In [1]:
import random
from datetime import datetime, timedelta
import io
import numpy as np
import heapq
import json
import operator
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from stop_words import get_stop_words
import json

import numpy.random as nprnd

stop_words = get_stop_words('english')


In [2]:
path_to_data = '../data/'

##########################
# load files #                           
##########################

training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)
#training_info = pd.read_csv(path_to_data + 'training_info.csv', sep=',', header=0)
training_info = pd.read_csv(path_to_data+"training_info2.csv",sep=',', header=0, index_col=0)
test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)
#test_info = pd.read_csv(path_to_data + 'test_info.csv', sep=',', header=0)
test_info = pd.read_csv(path_to_data+"test_info2.csv",sep=',', header=0, index_col=0)

global sent_to
with io.open('../data/sent_to.json') as json_data:
    sent_to = json.load(json_data)

global received_from
with io.open('../data/received_from.json') as json_data:
    received_from = json.load(json_data)

In [3]:
# Correct dates and put datetime format
# We do that because we noticed test_set is only composed of email posterior to the ones of train_set. 
# Datetime format allows to simulate posteriority in our train/test split
from datetime import datetime

for row in training_info.sort(['date']).iterrows():
    date = row[1]['date']
    if date[:3] == '000':
        date = '2' + date[1:]
        
    training_info.loc[row[0], 'date'] = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

for row in test_info.sort(['date']).iterrows():
    date = row[1]['date']
        
    test_info.loc[row[0], 'date'] = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')

/home/marc/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/home/marc/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


### Functions

In [18]:
def most_similar_sklearn(array_embedding_sparse, mail_tfidf, n):
    
    similarities = cosine_similarity(array_embedding_sparse, mail_tfidf)
    if int(round(sorted(similarities[:,0], reverse=True)[0])) ==1:
        closest_ids = similarities[:,0].argsort()[::-1][1:]
    else:
        closest_ids = similarities[:,0].argsort()[::-1]
    
    return closest_ids[:n], similarities

def get_sender(query_mid, training):
    for row in training.iterrows():
        mids = row[1]['mids'].split()
        for mid in mids:
            if int(mid) == query_mid:
                sender = row[1]['sender']
                break
    return sender


def get_10_recipients(closest_ids_per_sender, training_info, similarities):
    dic_of_recipients = {}
    #weight = len(closest_ids_per_sender)+1
    for idx in closest_ids_per_sender:
        recipients = training_info.loc[idx,'recipients'].split()
        for recipient in recipients:
            if '@' in recipient:
                dic_of_recipients[recipient] = dic_of_recipients.get(recipient, 0) + similarities[idx][0]

    return dic_of_recipients

def get_recency_features(X_train_info_sender, mail_date, n_recency_features):    
    dic_recency = {}
    df_last_sent_emails = X_train_info_sender[X_train_info_sender.date<= mail_date].sort_values(by = 'date', ascending = False)[:n_recency_features]
    for row in df_last_sent_emails.iterrows():
        recipients = row[1]['recipients'].split()
        for recipient in recipients:
            if '@' in recipient:
                dic_recency[recipient] = dic_recency.get(recipient, 0) + 1
    norm = sum(dic_recency.values())
    for k,v in dic_recency.iteritems():
        dic_recency[k] = float(v)/norm
    
    return dic_recency

def mean_ap(suggested_10_recipients, ground_truth):
    MAP = 0
    correct_guess = 0
    for i, suggestion in enumerate(suggested_10_recipients):
        if suggestion in ground_truth:
            correct_guess +=1
            MAP += float(correct_guess)/(i+1)
    MAP = float(MAP)/min(10, len(ground_truth))
    return MAP

def header_address_ressemblance(text, address):
    head = text[:20].lower()
    name = address[:address.index('@')].split('.')
    for n in name:
        if len(n)>2:
            if n in head:
                return True
    return False

def generate_features(mail_tfidf, mail_date, ground_truth, sender, n, n_recency_features, mail_header):
    
    X_train_info_sender = X_train_info[X_train_info.sender == sender]
    index_sender = X_train_info_sender.index.values
    X_train_info_sender.index = range(X_train_info_sender.shape[0])
    array_embedding_sparse_sender = array_embedding_sparse[index_sender]

    closest_ids_per_sender, similarities = most_similar_sklearn(array_embedding_sparse_sender, mail_tfidf, n)

    dic_recency = get_recency_features(X_train_info_sender, mail_date, n_recency_features)

    dic_of_recipients = get_10_recipients(closest_ids_per_sender, X_train_info_sender, similarities)

    new_features_per_mail = np.zeros((len(dic_of_recipients), 5))
    labels_per_mail = np.zeros((len(dic_of_recipients), 1))
    index = 0
    for k,v in dic_of_recipients.iteritems():
        KNNScore = v
        NSF = sent_to[sender][k]
        NRF = 0
        if sender in received_from.keys():
            NRF = received_from[sender].get(k, 0)

        recency = 0
        if k in dic_recency.keys():
            recency = dic_recency[k]
        
        if ground_truth != None:
            if k in ground_truth:
                labels_per_mail[index, :] = 1
        
        head = 1.0 * header_address_ressemblance(mail_header, k)

        new_features_per_mail[index, :] = [KNNScore, NSF, NRF, recency,head]
        index +=1

    return new_features_per_mail, labels_per_mail, dic_of_recipients

## Create Features

In [5]:
#Declare Global variables:
global X_train_info
global X_test_info
global array_embedding_sparse

In [6]:
submission = False
training_info = training_info.sort_values(by='date')

if submission:
    # submission procedure
    X_train_info = training_info
    X_test_info = test_info
    
else:
    # test procedure
    split_date=datetime(2001, 8, 25)
    X_train_info = training_info[training_info.date <= split_date]
    #X_test_info = training_info[training_info.date > split_date]
    
    #Randomize selection of test set:
    X_test_info = training_info[training_info.date > split_date]
    mask = nprnd.choice(range(X_test_info.shape[0]), size=1000, replace=False)
    X_test_info.index = range(X_test_info.shape[0])
    X_test_info = X_test_info[X_test_info.index.isin(mask)]

    X_train_info = training_info

In [7]:
if submission:
    tfidf = TfidfVectorizer(stop_words = stop_words)
    array_embedding_sparse = tfidf.fit_transform(np.concatenate((X_train_info['body'].values,X_test_info['body'].values)))
    array_embedding_sparse = array_embedding_sparse[:X_train_info.shape[0]]
else:
    #With porter stemming:
    #tfidf = TfidfVectorizer(tokenizer= tokenize, stop_words = stop_words)
    #Without stemming:
    tfidf = TfidfVectorizer(stop_words = stop_words)
    array_embedding_sparse = tfidf.fit_transform(X_train_info['body'].values)

## Compute New features

In [17]:
new_features_all = np.zeros((0,4))
labels_all = np.zeros((0,1))

# number of closest neighbors to collect recipients from:
n = 30
n_recency_features = 50

#re-arrange train index
X_train_info.index = range(X_train_info.shape[0])

t_all = datetime.now()
t_100 = datetime.now()

count = 1
query_id = 15000

for query_id in X_train_info.index.values:
    count+=1
    if count%100==0:
        print count
        print datetime.now()-t_100
        t_100 = datetime.now()

    # Get info on considered mail
    mail = X_train_info['body'][query_id]
    header_address_ressemblance(mail, sender)
    mail_tfidf = tfidf.transform([mail])
    mail_date = X_train_info['date'][query_id]
    ground_truth = X_train_info['recipients'][query_id].split()
    sender = X_train_info['sender'][query_id]

    # Compute Features For this email
    new_features_per_mail, labels_per_mail, dic_of_recipients = generate_features(mail_tfidf, mail_date, ground_truth, sender, n, n_recency_features, mail[:20])

    # Add to global features
    new_features_all = np.concatenate((new_features_all, new_features_per_mail))
    labels_all = np.concatenate((labels_all, labels_per_mail))


print "total took:", datetime.now()-t_all

george
george
mark
john
john
mark
tracy
tracy
rod
susan
susan
mark
sara
mark
per
dale
100
0:00:07.120522
susan
susan


KeyboardInterrupt: 

In [255]:
t1 = datetime.now()
#np.save('../data/new_features', new_features_all)
#np.save('../data/labels', labels_all)
print datetime.now() - t1

0:00:00.120710


In [73]:
new_features_all = np.load('../data/new_features1.npy')
labels_all = np.ravel(np.load('../data/labels1.npy'))

## Train Algorithm

In [74]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
#SVM = LinearSVC(dual=False)
#SVM.fit(new_features_all, labels_all)

# C parameter: smaller values mean bigger regularization
#LR = LogisticRegression(C=0.01)
#LR.fit(new_features_all, labels_all)

ABC = AdaBoostClassifier()
ABC.fit(new_features_all, labels_all)

## Test

Create test features

In [67]:
n = 30
n_recency_features = 50
#re-arrange train index
X_train_info.index = range(X_train_info.shape[0])
t_all = datetime.now()
t_100 = datetime.now()
results = pd.DataFrame(columns=['recipients'])
results.index.name = 'mid'
all_mean_ap = []
all_ground_truth = []
all_suggestions = []

count=0
for query_id in X_test_info.head(2).index.values:

    count+=1
    if count%100==0:
        print count
        print datetime.now()-t_100
        t_100 = datetime.now()

    mail = X_test_info['body'][query_id]
    mail_tfidf = tfidf.transform([mail])
    mail_date = X_test_info['date'][query_id]
    if submission:
        ground_truth = None
        query_mid = X_test_info['mid'][query_id]
    else:
        ground_truth = X_test_info['recipients'][query_id].split()
    sender = X_test_info['sender'][query_id]
    
    # Compute Features For this email
    new_features_per_mail, labels_per_mail, dic_of_recipients = generate_features(mail_tfidf, mail_date, ground_truth, sender, n, n_recency_features)
    
    # Once the features are computed, we can predict the 10 recipients
    order = ABC.predict_proba(new_features_per_mail)[:,1].argsort()[::-1]
    recipients = np.array(dic_of_recipients.keys())
    suggested_10_recipients = recipients[order][:10]

    if submission:
        string_recipients = ''
        for k in suggested_10_recipients:
            string_recipients+=k + ' '
        results.loc[query_mid, 'recipients'] = string_recipients
    else:

        all_suggestions.append(suggested_10_recipients)
        ground_truth.append(ground_truth)
        all_mean_ap.append(mean_ap(suggested_10_recipients, ground_truth))


print "total took:", datetime.now()-t_all

total took: 0:00:00.172634


In [49]:
ground_truth

['mark.palmer@enron.com',
 'richard.shapiro@enron.com',
 'steven.j.kean@enron.com',
 'mark.koenig@enron.com',
 'michael.tribolet@enron.com']

In [59]:
v

0.33643196520477042

In [50]:
mail

'I wanted IR and PR to be aware that within the next few days  the PG&E Bankruptcy team will file with the Judge Enron s current claim.  We intend to ask the Judge to Lift the Stay so that we can file at the CPUC a complaint to recoup our current receivable.  I m not sure what our public statements have been and as soon as we prepare the filing, I ll let you know.If anyone has any questions, please give me a call.Thanks,Jim'

In [53]:
for idx in closest_ids_per_sender:
    print X_train_info_sender[X_train_info_sender.index == idx]['recipients'].values[0]
    print 

e..haedicke@enron.com richard.b.sanders@enron.com richard.shapiro@enron.com steven.j.kean@enron.com karen.denne@enron.com lisa.mellencamp@enron.com mark.palmer@enron.com vicki.sharp@enron.com c..williams@enron.com dan.leff@enron.com

e..haedicke@enron.com richard.b.sanders@enron.com richard.shapiro@enron.com steven.j.kean@enron.com karen.denne@enron.com lisa.mellencamp@enron.com mark.palmer@enron.com vicki.sharp@enron.com c..williams@enron.com dan.leff@enron.com

e..haedicke@enron.com richard.b.sanders@enron.com richard.shapiro@enron.com steven.j.kean@enron.com karen.denne@enron.com lisa.mellencamp@enron.com mark.palmer@enron.com vicki.sharp@enron.com c..williams@enron.com dan.leff@enron.com

c..williams@enron.com peggy.mahoney@enron.com mark.koenig@enron.com richard.b.sanders@enron.com

c..williams@enron.com peggy.mahoney@enron.com mark.koenig@enron.com richard.b.sanders@enron.com

amr.ibrahim@enron.com

susan.mara@enron.com linda.noske@enron.com harry.kingerski@enron.com paul.kaufman

In [54]:
dic_of_recipients

{'alan.comnes@enron.com': 0.82014060600854344,
 'amr.ibrahim@enron.com': 0.19292907510461701,
 'bhansen@lhom.com': 0.17167940519796948,
 'c..williams@enron.com': 1.4914341994999485,
 'chris.foster@enron.com': 0.33643196520477042,
 'christi.nicolay@enron.com': 0.58488662503206501,
 'christian.yoder@enron.com': 0.33643196520477042,
 'christopher.calger@enron.com': 0.33643196520477042,
 'cynthia.sandherr@enron.com': 0.33643196520477042,
 'dan.leff@enron.com': 1.3486754531772487,
 'dan.staines@enron.com': 0.42839949983756281,
 'daniel.allegretti@enron.com': 0.42839949983756281,
 'david.w.delainey@enron.com': 0.68238752312152096,
 'dennis.benevides@enron.com': 0.49780268987988519,
 'diann.huddleson@enron.com': 0.2739588836512068,
 'don.black@enron.com': 0.33643196520477042,
 'donna.fulton@enron.com': 0.42839949983756281,
 'dwatkiss@bracepatt.com': 0.13740129554717412,
 'e..haedicke@enron.com': 0.66628793005572762,
 'elizabeth.sager@enron.com': 0.33643196520477042,
 'elizabeth.tilney@enron.c

In [56]:
dic_recency

{'alan.comnes@enron.com': 0.008695652173913044,
 'aleck.dadson@enron.com': 0.017391304347826087,
 'amr.ibrahim@enron.com': 0.017391304347826087,
 'c..williams@enron.com': 0.02608695652173913,
 'dan.staines@enron.com': 0.017391304347826087,
 'daniel.allegretti@enron.com': 0.017391304347826087,
 'dave.perrino@enron.com': 0.008695652173913044,
 'david.howe@enron.com': 0.008695652173913044,
 'diann.huddleson@enron.com': 0.008695652173913044,
 'dolores.muzzy@enron.com': 0.008695652173913044,
 'donna.fulton@enron.com': 0.008695652173913044,
 'elizabeth.linnell@enron.com': 0.008695652173913044,
 'eric.gadd@enron.com': 0.008695652173913044,
 'ginger.dernehl@enron.com': 0.008695652173913044,
 'gus.perez@enron.com': 0.008695652173913044,
 'harry.kingerski@enron.com': 0.008695652173913044,
 'howard.fromer@enron.com': 0.017391304347826087,
 'j..noske@enron.com': 0.05217391304347826,
 'janel.guerrero@enron.com': 0.02608695652173913,
 'janine.migden@enron.com': 0.008695652173913044,
 'jeff.dasovich@

In [64]:
v

0.33643196520477042

In [69]:
labels_per_mail

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
      

In [68]:
np.set_printoptions(threshold=np.inf)
print new_features_per_mail

[[  2.19627512e+00   3.89797883e-02   0.00000000e+00   3.47826087e-02]
 [  3.46307345e-01   8.42155919e-04   0.00000000e+00   0.00000000e+00]
 [  1.61370725e-01   1.08277190e-03   0.00000000e+00   0.00000000e+00]
 [  1.92929075e-01   4.21077960e-03   4.07608696e-02   1.73913043e-02]
 [  6.82387523e-01   9.98556304e-03   0.00000000e+00   0.00000000e+00]
 [  3.87985841e-01   8.42155919e-04   0.00000000e+00   8.69565217e-03]
 [  7.64831465e-01   1.52791145e-02   0.00000000e+00   0.00000000e+00]
 [  2.70379924e-01   8.18094321e-03   0.00000000e+00   1.73913043e-02]
 [  1.37401296e-01   1.08277190e-03   0.00000000e+00   0.00000000e+00]
 [  4.28399500e-01   6.01539942e-04   0.00000000e+00   0.00000000e+00]
 [  4.28399500e-01   1.44369586e-03   0.00000000e+00   1.73913043e-02]
 [  3.36431965e-01   4.09047161e-03   0.00000000e+00   0.00000000e+00]
 [  1.41990330e-01   2.64677575e-03   0.00000000e+00   8.69565217e-03]
 [  2.35511194e+00   6.62897016e-02   9.85054348e-02   5.21739130e-02]
 [  1.

In [63]:
np.set_printoptions(threshold=np.inf)
print new_features_per_mail

[[  2.19627512e+00   3.89797883e-02   0.00000000e+00   3.47826087e-02]
 [  3.46307345e-01   8.42155919e-04   0.00000000e+00   0.00000000e+00]
 [  1.61370725e-01   1.08277190e-03   0.00000000e+00   0.00000000e+00]
 [  1.92929075e-01   4.21077960e-03   4.07608696e-02   1.73913043e-02]
 [  6.82387523e-01   9.98556304e-03   0.00000000e+00   0.00000000e+00]
 [  3.87985841e-01   8.42155919e-04   0.00000000e+00   8.69565217e-03]
 [  7.64831465e-01   1.52791145e-02   0.00000000e+00   0.00000000e+00]
 [  2.70379924e-01   8.18094321e-03   0.00000000e+00   1.73913043e-02]
 [  1.37401296e-01   1.08277190e-03   0.00000000e+00   0.00000000e+00]
 [  4.28399500e-01   6.01539942e-04   0.00000000e+00   0.00000000e+00]
 [  4.28399500e-01   1.44369586e-03   0.00000000e+00   1.73913043e-02]
 [  3.36431965e-01   4.09047161e-03   0.00000000e+00   0.00000000e+00]
 [  1.41990330e-01   2.64677575e-03   0.00000000e+00   8.69565217e-03]
 [  2.35511194e+00   6.62897016e-02   9.85054348e-02   5.21739130e-02]
 [  1.

In [ ]:
# Once the features are computed, we can predict the 10 recipients
order = ABC.predict_proba(new_features_per_mail)[:,1].argsort()[::-1]
recipients = np.array(dic_of_recipients.keys())
suggested_10_recipients = recipients[order][:10]

if submission:
    string_recipients = ''
    for k in suggested_10_recipients:
        string_recipients+=k + ' '

    results.loc[query_mid, 'recipients'] = string_recipients
else:

    all_suggestions.append(suggested_10_recipients)
    ground_truth.append(ground_truth)
    all_mean_ap.append(mean_ap(suggested_10_recipients, ground_truth))

In [109]:
results.to_csv('../submission/learning_first_try.csv')

In [103]:
np.mean(all_mean_ap)

0.3652579746157722

## Get NSF and NRF
- NSF: number of messages sent by sender to this recipient, divided by all messages sent by sender 
- NRF: number of messages received from the recipient to the sender, divided by all messages received by sender

In [7]:
sent_to = {}
for row in training.iterrows():
    sender = row[1]['sender']
    sent_to[sender] = {}
    for mid in row[1]['mids'].split():
        recipients = training_info[training_info.mid == int(mid)]['recipients'].values[0].split()
        for recipient in recipients:
            if '@' in recipient:
                if recipient in sent_to[sender].keys():
                    sent_to[sender][recipient] += 1
                else:
                    sent_to[sender][recipient] = 1

In [8]:
received_from = {}
for sender in sent_to.keys():
    for recipient in sent_to[sender].keys():
        if recipient not in received_from.keys():
            received_from[recipient] = {}
        if sender not in received_from[recipient].keys():
            received_from[recipient][sender] = sent_to[sender][recipient]
        else:
            received_from[recipient][sender] += sent_to[sender][recipient]

In [9]:
# normalization:
for sender in sent_to.keys():
    norm = sum(sent_to[sender].values())
    for recipient in sent_to[sender].keys():
        sent_to[sender][recipient] = float(sent_to[sender][recipient])/norm

In [10]:
# normalization:
for recipient in received_from.keys():
    norm = sum(received_from[recipient].values())
    for sender in received_from[recipient].keys():
        received_from[recipient][sender] = float(received_from[recipient][sender])/norm

In [13]:
with io.open('../data/sent_to.json', 'w', encoding='utf-8') as f:
    f.write(unicode(json.dumps(sent_to, ensure_ascii=False)))
with io.open('../data/received_from.json', 'w', encoding='utf-8') as f:
    f.write(unicode(json.dumps(received_from, ensure_ascii=False)))

In [15]:
with io.open('../data/sent_to.json') as json_data:
    sent_to = json.load(json_data)